In [5]:
%matplotlib inline
import os
import math 
import numpy as np
import pandas as pd 
import seaborn as sns
import random
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from scipy.stats import pearsonr, spearmanr, gaussian_kde
from sklearn.decomposition import TruncatedSVD, PCA
from matplotlib.colors import rgb2hex
pd.options.display.max_rows = 2000
pd.options.display.max_columns = 999

sns.set_style('white')
ahba_dir     = '/scr/malta3/workspace/project_iron/AHBA'
ahba_dir     = '/Users/kanaan/Google Drive/TS-EUROTRAIN/RESULTS_QSMv3/SEPT10/AHBA/'
ahba_dir2     = '/Users/kanaan/Google Drive/TS-EUROTRAIN/RESULTS_QSMv3/AUG27/AHBA/'
save_fig_dir = '/Users/kanaan/Google Drive/TS-EUROTRAIN/Papers/2016_QSM_paper/Figures_python_v2'

drop_strings = ['coords_native', 'donor_names', 'struct_id', 
                'struct_name', 'top_struct', 'Mean', 'Median', 
                'PC1', 'PC2','PC3', #'SVD1g', 'SVD2g','SVD3g', 'SVD3p', 'SVD2p','SVD3p', 
                'corrected_mni_x', 'corrected_mni_y', 'corrected_mni_z']

import warnings
warnings.filterwarnings('ignore')

# quality control
# drop wells outside the striatal masks 
drop_wells = [2371, 13913, 2379, 160091500]


# READ QSM statmap data 

In [2]:
#df_MNI = pd.read_csv(os.path.join(ahba_dir,'QSM_TSTATS/NIFTI_VALUES_permute_10K_SEPT20.csv'), index_col = 0 )
df_MNI = pd.read_csv(os.path.join(ahba_dir,'QSM_TSTATS/MNI_NIFTI_VALUES_permute_10K_OCT2.csv'), index_col = 0 )

# READ AHBA data (app. 15000 genes)

In [31]:
def return_expression_df(geneset):
    return pd.read_csv(os.path.join(ahba_dir, 'AHBA_%s.csv'%geneset),index_col=0)

# Concat ahba genes 
ahba_a = return_expression_df('18589_a').drop(drop_strings,axis =1)
ahba_b = return_expression_df('18589_b').drop(drop_strings,axis =1)
ahba_c = return_expression_df('18589_c').drop(drop_strings,axis =1)
ahba_d = return_expression_df('18589_d').drop(drop_strings,axis =1)
ahba_e = return_expression_df('18589_e').drop(drop_strings,axis =1)
ahba_f = return_expression_df('18589_f').drop(drop_strings,axis =1)
ahba_g = return_expression_df('18589_g')
AHBA   = pd.concat([ahba_a, ahba_b, ahba_c, ahba_d, ahba_e, ahba_f, ahba_g], axis =1)
#AHBA   = AHBA#.rename(columns={"struct_name": "struct"}) # Dont change struct name here.. this causes indexing problems
AHBA = AHBA.loc[:,~AHBA.columns.duplicated()]# drops duplicate columns

GENE_LIST = sorted(AHBA.columns[:-13])
print 'Total Number of AHBA Genes =', len(GENE_LIST)

#set([x for x in GENE_LIST if GENE_LIST.count(x) > 1])  # checkif there are duplicated columns 

Total Number of AHBA Genes = 14983


In [3]:
AHBA = pd.read_csv(os.path.join(ahba_dir, 'ahba_data', 'AHBA_20737.csv'), index_col = 0)
AHBA = AHBA.iloc[:,:-13]
GENE_LIST = AHBA.columns
print 'Total Number of AHBA Genes =', len(GENE_LIST)


Total Number of AHBA Genes = 20737


In [6]:
def make_chi_gene_correleation(df_ahba, df_chi, roi):
    
    
    df = pd.DataFrame(index=GENE_LIST, columns=['r_val', 'p_val'])    
    
    for gene in GENE_LIST:
        if df_ahba[gene] is not None:
            dfx = pd.DataFrame(index=df_ahba.index)
            dfx[gene]  = df_ahba[gene] 
            dfx[roi]   = df_chi[roi]
            dfx = dfx.drop(drop_wells,axis=0).dropna()

            r_val , p_val= pearsonr(dfx[gene], dfx[roi])
            #print gene, r_val, p_val

            df.loc[gene]['r_val'] = float(r_val)
            df.loc[gene]['p_val'] = float(p_val) 
        
    return df.sort_values('r_val', ascending=0)
        

In [7]:
corr_str3m = make_chi_gene_correleation(AHBA, df_MNI, 'STR3_MOTOR_tstat_CP_1mm')

In [10]:
corr_str = make_chi_gene_correleation(AHBA, df_MNI, 'STR_tstat_CP_1mm')

In [26]:
corr_str_av = make_chi_gene_correleation(AHBA, df_MNI, 'MEAN_STR_ALL_2mm')

In [32]:
corr_gm_av = make_chi_gene_correleation(AHBA, df_MNI, 'MEAN_ALL_GM_2')

In [28]:
corr_pal_av = make_chi_gene_correleation(AHBA, df_MNI, 'MEAN_ALL_PALL_3')


In [13]:
for i in corr_str[0:100].index:
    print i 

ZNFX1-AS1
AC073346.1
HHLA3
CCL4
RPL23
LOC100287590
RP11-122A3.2
SHB
KIF12
C9orf123
RHCE
ZDHHC11
NME2P1
ASB9
SNHG1
PIR
CHCHD5
PPDPF
CCL3L1
CIDEA
KLRC2
IL32
CCL3
AGPHD1
H1F0
KLKB1
CRYBB2
MAD1L1
MYL5
NUP85
UQCR11
PXMP4
CACNA2D4
USF1
PCDH17
STARD5
SPATA7
TTLL13
RRP7B
NME2
RP11-115C21.2
YIF1B
CRYBB2P1
PLEKHN1
UQCRBP1
UCRC
C14orf156
SNRPE
RPL13AP6
ODF4
JOSD2
SNHG8
ZNF557
C16orf59
BRK1
CDON
NDUFA1
OR11H12
MATN1
LOC729458
RPS18
MYBPC3
GHRL
MRPL33
RXFP2
KIAA1161
C2orf3
RNF7
OR7E5P
C17orf89
ATG7
UQCRB
GPR116P1
LINC00324
RPL37
XAB2
RPL39
LOC100131972
MX1
SEC14L5
PHKA2
PRSS45
LENG1
CCDC85B
NLRP11
CISD3
MAGI2
LOC730144
C11orf51
C8orf4
METTL2B
FAM118A
SEPN1
EIF3B
ZMAT2
PFDN5
TTF2
IGFBP6
GGTA1
MMD


In [12]:
corr_str[0:200]

,r_val,p_val
ZNFX1-AS1,0.393464,5.12644e-05
AC073346.1,0.385352,7.52303e-05
HHLA3,0.38387,8.06031e-05
CCL4,0.377551,0.000107778
RPL23,0.372263,0.00013682
LOC100287590,0.369342,0.000155819
RP11-122A3.2,0.36816,0.00016418
SHB,0.364684,0.000191239
KIF12,0.363849,0.00019833
C9orf123,0.363262,0.000203452
